In [11]:
import urllib.request as ur # import para obtener la página web 
import re                   # import para utilizar expresiones regulares 

# Obtener página web con incidencias de Madrid 
f = ur.urlopen('http://infocar.dgt.es/etraffic/Incidencias?ca=13&provIci=&caracter=acontecimiento&accion_consultar=Consultar&IncidenciasRETENCION=IncidenciasRETENCION&IncidenciasPUERTOS=IncidenciasPUERTOS&IncidenciasMETEOROLOGICA=IncidenciasMETEOROLOGICA&IncidenciasEVENTOS=IncidenciasEVENTOS&IncidenciasOTROS=IncidenciasOTROS&IncidenciasRESTRICCIONES=IncidenciasRESTRICCIONES&ordenacion=fechahora_ini-DESC')
s = f.read().decode()
f.close()


In [12]:
# Método para eliminar tags html que no nos interesan con ER 
def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr,'',raw_html)
    return cleantext

In [13]:
from pprint import pprint     # import para hacer prints boninitos 

# Obtener las incidencias que tienen hora fin 
print("Incidencias con hora fin")
confin = re.findall("<tr.*?<td.*?class='orange'>(\d\d:\d\d).*?(\d\d/\d\d/\d\d\d\d).*?<td.*?(\d\d:\d\d).*?(\d\d/\d\d/\d\d\d\d).*?<td.*?<td.*?<td.*?<b>(.*?)</b>.*?<td.*?<span.*?<b>(.*?)</b></span>.*?</td>.*?",s,re.DOTALL)
pprint(confin)

# Obtener todas las incidencias 
print("\nTodas las incidencias")
conjunto = re.findall("<tr.*?<td.*?class='orange'>(\d\d:\d\d).*?(\d\d/\d\d/\d\d\d\d).*?<td.*?(\d\d:\d\d)?.*?(\d\d/\d\d/\d\d\d\d)?.*?<td.*?<td.*?<td.*?<b>(.*?)</b>.*?<td.*?<span.*?<b>(.*?)</b></span>.*?</td>.*?",s,re.DOTALL)
pprint(conjunto)



Incidencias con hora fin
[('21:00',
  '28/02/2019',
  '20:35',
  '28/02/2019',
  'MADRID',
  'RETENCIÓN / CONGESTIÓN</b> por <b>CIRCULACIÓN</b> con circulación '
  'irregular  en<b></b>: </span> <span style="margin-top:10px; float:left; '
  'clear:both">  - La AUTOPISTA / AUTOVÍA <b> <span style="color:#ab3000">A-2 '
  '</span> </b> a la altura de <b>MADRID (MADRID)</b> desde el <b> km 4.5</b> '
  'al <b> km 7</b> sentido <b>DECRECIENTE DE LA KILOMETRACIÓN</b> hacia '
  '<b>MADRID'),
 ('20:05',
  '28/02/2019',
  '17:33',
  '28/02/2019',
  'MADRID',
  'RETENCIÓN / CONGESTIÓN</b> por <b>CIRCULACIÓN</b> con circulación '
  'irregular  en<b></b>: </span> <span style="margin-top:10px; float:left; '
  'clear:both">  - La AUTOPISTA / AUTOVÍA <b> <span style="color:#ab3000">A-1 '
  '</span> </b> a la altura de <b>MADRID (MADRID)</b> desde el <b> km 9</b> al '
  '<b> km 12</b> sentido <b>DECRECIENTE DE LA KILOMETRACIÓN</b> hacia '
  '<b>MADRID'),
 ('17:01',
  '28/02/2019',
  '15:31',
  '28/02/2

In [10]:
diccionario = {} # variable que se utilizará para almacenar las incidencias una sola vez 

for x in range(len(confin)):
    identificador = confin[x][0] + confin[x][1] + confin[x][4]  # concatenación de valores para crear identificador para diccionario
    diccionario.update({identificador : confin[x]}) # agregación de valores al diccionario

for x in range(len(conjunto)):
    identificador = conjunto[x][0] + conjunto[x][1] + conjunto[x][4] # concatenación de valores para crear identificador para diccionario
    if diccionario.get(identificador) == None:    # En caso de que el identificador exista, salta este paso
        diccionario.update({identificador : conjunto[x]}) # Si el identificador no existe, se crea un nuevo elemento en el diccionario
'''
Para todas las carreteras
'''
        
for x in diccionario:
    valores = diccionario.get(x) # obtener el valor correspondiente para el identificador 
    fin = "" # declarar variable para mensaje en caso de que la incidencia tenga hora fin
    cleantext =  cleanhtml(valores[5]) # quitar las etiquetas de html del texto que describe la incidencia 
    if valores[2] != '':  # En caso de que exista una hora fin
        mensaje = "Hubo un atasco en la carretera " + valores[4]
        fin = " el cual terminó a las " + valores[2] + " el día " + valores[3]
    else: 
        mensaje = "Existe un atasco en la carretera " + valores[4]
    mensaje = mensaje + " que inicio a las " + valores[0] + " el día " + valores[1]
    mensaje = mensaje + fin
    mensaje = mensaje + " con la siguientes características:\n" +  cleantext
    
    print(mensaje+"\n")
    


Hubo un atasco en la carretera MADRID que inicio a las 21:00 el día 28/02/2019 el cual terminó a las 20:35 el día 28/02/2019 con la siguientes características:
RETENCIÓN / CONGESTIÓN por CIRCULACIÓN con circulación irregular  en:    - La AUTOPISTA / AUTOVÍA  A-2   a la altura de MADRID (MADRID) desde el  km 4.5 al  km 7 sentido DECRECIENTE DE LA KILOMETRACIÓN hacia MADRID

Hubo un atasco en la carretera MADRID que inicio a las 20:05 el día 28/02/2019 el cual terminó a las 17:33 el día 28/02/2019 con la siguientes características:
RETENCIÓN / CONGESTIÓN por CIRCULACIÓN con circulación irregular  en:    - La AUTOPISTA / AUTOVÍA  A-1   a la altura de MADRID (MADRID) desde el  km 9 al  km 12 sentido DECRECIENTE DE LA KILOMETRACIÓN hacia MADRID

Hubo un atasco en la carretera MADRID que inicio a las 17:01 el día 28/02/2019 el cual terminó a las 15:31 el día 28/02/2019 con la siguientes características:
RETENCIÓN / CONGESTIÓN por CIRCULACIÓN con circulación irregular  en:    - La AUTOPISTA /

In [15]:
'''
Para la carretera M-40 
'''

for x in diccionario:
    if "M-40" in x: 
        valores = diccionario.get(x) # obtener el valor correspondiente para el identificador 
        fin = "" # declarar variable para mensaje en caso de que la incidencia tenga hora fin
        cleantext =  cleanhtml(valores[5]) # quitar las etiquetas de html del texto que describe la incidencia 
        if valores[2] != '':  # En caso de que exista una hora fin
            mensaje = "Hubo un atasco en la carretera " + valores[4]
            fin = " el cual terminó a las " + valores[2] + " el día " + valores[3]
        else: 
            mensaje = "Existe un atasco en la carretera " + valores[4]
        mensaje = mensaje + " que inicio a las " + valores[0] + " el día " + valores[1]
        mensaje = mensaje + fin
        mensaje = mensaje + " con la siguientes características:\n" +  cleantext

        print(mensaje+"\n")
    


Hubo un atasco en la carretera M-111 que inicio a las 06:00 el día 28/02/2019 el cual terminó a las 23:00 el día 28/02/2019 con la siguientes características:
RESTRICCIONES A MERCANCÍAS PELIGROSAS por CIRCULACIÓN en Túnel Barajas:    -  La carretera   M-111   a la altura de MADRID (MADRID) desde el  km 1.1 al  km 5 sentido AMBOS SENTIDOS

